# Dryad API

# Setup

## Instructions

This notebook utilizes the Data Dryad API. No API Token is required.

## Additional Information

Documentation Guide:
- Dryad API ([Dryad](https://datadryad.org/api/v2/docs/))
- Dryad API ([GitHub](https://github.com/CDL-Dryad/dryad-app/tree/main/documentation/apis))

## Imports

In [1]:
import requests # For querying data from API
import pandas as pd # For storing/manipulating query data
from tqdm import tqdm # Gives status bar on loop completion
from collections import OrderedDict
from flatten_json import flatten

# For loading credentials
import pickle
import os

# Data Wrangling

## Setup

In [2]:
# Search constants
BASE_URL = 'https://datadryad.org/api/v2'

## Extracting

In [3]:
def get_all_search_outputs(search_terms, flatten_output=False):
    """
    Call the Data Dryad API for each search term. 
    Results are retured in results['({term},)'] = df
    
    Params:
    - search_terms : list-like 
        collection of search terms to query over
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output
    
    Returns:
    - results : dict
        dictionary consisting of returned DataFrames from get_search_output for each query
    """
    
    results = OrderedDict()

    for search_term in search_terms:
        results[(search_term,)] = get_individual_search_output(search_term, flatten_output)
        
    return results

In [4]:
def _conduct_search_over_pages(search_url, search_params, flatten_output=False, delim=None):
    # Make sure proper delim passed in
    if delim:
        assert isinstance(delim, str), 'Incorrect delim parameter passed in. Must be of type str'
    
    search_df = pd.DataFrame()
    
    # Perform initial search & convert results to json
    response = requests.get(search_url, params=search_params)
    output = response.json()
    
    # Loops over the search as long as the page was not empty
    while output.get('count'):
        # Extract relevant output data
        output = output['_embedded']
        
        if delim:
            output = output[delim]
        
        # Flatten output if necessary
        if flatten_output:
            output = [flatten(result) for result in output]
        else:
            output = output
        
        output_df = pd.DataFrame(output)
        output_df['page'] = search_params['page']
        
        search_df = pd.concat([search_df, output_df]).reset_index(drop=True)
        
        # Increment the page number to perform another search
        search_params['page'] += 1
        
        # Perform next search and convert results to json
        response = requests.get(search_url, params=search_params)
        output = response.json()

    return search_df

In [5]:
def get_individual_search_output(search_term, flatten_output=False):
    """
    Returns a list of all datasets available from the Data Dryad API
    
    Params:
    - search_term : str
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output
        
    Returns:
    - pandas.DataFrame
        DataFrame containing the output of the search query
    """
    
    # Set search params
    search_url = f'{BASE_URL}/search'
    start_page = 1
    page_size = 100
    
    search_params = {
        'q': search_term,
        'page': start_page,
        'per_page': page_size
    }
    
    return _conduct_search_over_pages(search_url, search_params, flatten_output, delim='stash:datasets')

### Run initial API query functions

In [6]:
search_terms = ['\"machine learning\" OR \"artificial intelligence\"', 
                '\"machine learning\"', 
                '\"artificial intelligence\"',
                '\"deep learning\"',
                '\"neural network\"',
                '\"supervised learning\"',
                '\"unsupervised learning\"',
                '\"reinforcement learning\"',
                '\"training data\"']

In [7]:
search_output_dict = get_all_search_outputs(search_terms, flatten_output=True)

#### Take a look at the results

In [ ]:
search_output_dict[('machine learning',)].head()

## Metadata

In [ ]:
def get_query_metadata(object_paths, flatten_output=False):
    """
    Retrieves the metadata for the file/files listed in object_paths
    
    Params:
    - object_paths : str/list-like
        string or list of strings containing the paths for the objects
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output
    
    Returns:
    - metadata_df : pandas.DataFrame
        DataFrame containing metadata for the requested dataset
    """
    
    # If a singular search term is provided as a string, need to wrap it in a list
    if type(object_paths) == str:
        object_paths = [object_paths]
    
    # Make sure our input is valid
    assert len(object_paths) > 0, 'Please enter at least one object id'
    
    # Set search variables
    start_page = 1
    metadata_df = pd.DataFrame()
    
    # Request the metadata for each object
    for object_path in tqdm(object_paths):
        # Set search variables
        search_url = f'{BASE_URL}/versions/{object_path}/files'
        search_params = {'page': start_page}
        
        # Conduct search
        object_df = _conduct_search_over_pages(search_url, search_params, flatten_output, delim='stash:files')

        # Add relevant data to DataFrame and merge
        object_df['id'] = object_path
        object_df['page'] = search_params['page']
        metadata_df = pd.concat([metadata_df, object_df]).reset_index(drop=True)
    
    return metadata_df

In [ ]:
def get_all_metadata(search_output_dict, flatten_output=False):
    """
    Retrieves all of the metadata that relates to the provided DataFrames
    
    Params:
    - search_output_dict : dict
        Dictionary of DataFrames from get_all_search_outputs
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output  
      
    Returns:
    - metadata_dict : collections.OrderedDict
        OrderedDict of DataFrames with metadata for each query
        Order matches the order of search_output_dict
    """
    
    ## Extract IDs from DataFrame, and returns as list of strings
    metadata_dict = OrderedDict()

    for query, df in search_output_dict.items():
        search_term = query[0]
        print(f'Retrieving {search_term} metadata')
        
        # Create object paths
        object_paths = df.id.convert_dtypes(convert_string=True).tolist()

        metadata_dict[query] = get_query_metadata(object_paths, flatten_output)
    
    return metadata_dict

### Run metadata API query functions

In [ ]:
metadata_dict = get_all_metadata(search_output_dict, flatten_output=True)

#### Take a look at the results

In [ ]:
metadata_dict[('machine learning',)].to_csv('dryad__metadata')

## Merge results

In [ ]:
def merge_search_and_metadata_dicts(search_dict, metadata_dict, on=None, left_on=None, right_on=None, save=False):
    """
    Merges together all of the search and metadata DataFrames by the given 'on' key
    
    Params:
    - search_dict (dict): dictionary of search output results
    - metadata_dict (dict): dictionary of metadata results
    - on (str/list-like): column name(s) to merge the two dicts on
    - left_on (str/list-like): column name(s) to merge the left dict on
    - right_on (str/list-like): column name(s) to merge the right dict on
    - save=False, optional (bool/list-like): specifies if the output DataFrames should be saved
        If True: saves to file of format 'data/kaggle/kaggle_{search_term}_{search_type}.csv'
        If list-like: saves to respective location in list of save locations
            Must contain enough strings (one per query; len(search_terms) * len(search_types))
            
    Returns:
    - df_dict (OrderedDict): OrderedDict containing all of the merged search/metadata dicts
    """

    # Make sure the dictionaries contain the same searches
    assert search_dict.keys() == metadata_dict.keys(), 'Dictionaries must contain the same searches'
    
    num_dataframes = len(search_dict)
    
    # Ensure the save variable data is proper
    try:
        if isinstance(save, bool):
            save = [save] * num_dataframes
        assert len(save) == num_dataframes
    except:
        raise ValueError('Incorrect save value(s)')
        
    # Merge the DataFrames
    df_dict = OrderedDict()
    for (query_key, search_df), (query_key, metadata_df), save_loc in zip(search_dict.items(), 
                                                                          metadata_dict.items(), 
                                                                          save):

        # Merge small version of "full" dataframe with "detailed" dataframe
        df_all = pd.merge(search_df, metadata_df, on=on, left_on=left_on, right_on=right_on, how='outer')
            
        # Save DataFrame
        if save_loc:
            data_dir = os.path.join('data', 'dryad')
            if isinstance(save_loc, str):
                output_file = save_loc
            elif isinstance(save_loc, bool):
                # Ensure figshare directory is already created
                if not os.path.isdir(data_dir):
                    os.path.mkdir(data_dir)
                
                search_term, search_type = query_key
                output_file = f'{search_term}_{search_type}.csv'
            else:
                raise ValueError(f'Save type must be bool or str, not {type(save_loc)}')

            search_df.to_csv(os.path.join(data_dir, output_file), index=False)
        
        df_dict[query_key] = df_all
    
    return df_dict

In [ ]:
df_dict = merge_search_and_metadata_dicts(search_output_dict, metadata_dict, on='id')

In [ ]:
df_dict[('machine learning',)]